# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# Load Datasets

In [2]:
df = pd.read_csv("../datasets/final_data/FIFA 1990-2018 - Cleaned.csv")
df_teams = pd.read_csv("../datasets/final_data/world_cup_comparisons.csv")
fifa_ranking = pd.read_csv("../datasets/final_data/fifa_ranking.csv")


In [3]:
df=df.replace({"South Korea":"Korea Republic"})
df = df.replace({"Iran":"IR Iran"})
df = df.replace({"Soviet Union":"Russia"})
df = df.replace({"Czechoslovakia":"Czech Republic"})


df_teams=df_teams.replace({"South Korea":"Korea Republic"})
df_teams = df_teams.replace({"Iran":"IR Iran"})

fifa_ranking=fifa_ranking.replace({"South Korea":"Korea Republic"})
fifa_ranking = fifa_ranking.replace({"Iran":"IR Iran"})
# fifa_ranking = fifa_ranking.replace({"Czech Republic":"Czechoslovakia"})



In [4]:
df.head()

Team1         Team2       Rounds                   Stadium  \
0    Russia  Saudi Arabia  Group Phase          Luzhniki Stadium   
1     Egypt       Uruguay  Group Phase        Ekaterinburg Arena   
2   Morocco       IR Iran  Group Phase  Saint Petersburg Stadium   
3  Portugal         Spain  Group Phase             Fisht Stadium   
4    France     Australia  Group Phase               Kazan Arena   

             City   Time  Day  Month  Year  Score1  Score2  \
0          Moscow  18:00   14      6  2018       5       0   
1    Ekaterinburg  17:00   15      6  2018       0       1   
2  St. Petersburg  18:00   15      6  2018       0       1   
3           Sochi  21:00   15      6  2018       3       3   
4           Kazan  13:00   16      6  2018       2       1   

                                             Remarks  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3  {WinTeamName} win on aggregate after regular t...  
4                                                NaN

# Duplicate

In [5]:

# we should change all respective column for duplicate
for index,row in df.iterrows():
    df.set_value(index+488,["Team1","Team2","Rounds","Stadium","City","Time","Day","Month","Year","Score1","Score2","Remarks"],[row["Team2"],row["Team1"],row["Rounds"],row['Stadium'],row['City'],row['Time'],row["Day"],row["Month"],row["Year"],row["Score2"],row["Score1"],row['Remarks']])
df.shape


/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


(976, 12)

In [6]:
df.iloc[487]

Team1              Germany
Team2            Argentina
Rounds              Finals
Stadium    Stadio Olimpico
City                  Rome
Time                 20:00
Day                      8
Month                    7
Year                  1990
Score1                   1
Score2                   0
Remarks                NaN
Name: 487, dtype: object

In [7]:
df.iloc[975]

Team1            Argentina
Team2              Germany
Rounds              Finals
Stadium    Stadio Olimpico
City                  Rome
Time                 20:00
Day                      8
Month                    7
Year                  1990
Score1                   0
Score2                   1
Remarks                NaN
Name: 975, dtype: object

# Ranking


In [8]:
fifa_ranking.head(10)

rank         country_full country_abrv total_points  previous_points  \
0     1              Germany          GER          0.0             57.0   
1     2                Italy          ITA          0.0             57.0   
2     3          Switzerland          SUI          0.0             50.0   
3     4               Sweden          SWE          0.0             55.0   
4     5            Argentina          ARG          0.0             51.0   
5     6  Republic of Ireland          IRL          0.0             54.0   
6     7               Russia          RUS          0.0             52.0   
7     8               Brazil          BRA          0.0             55.0   
8     9               Norway          NOR          0.0             49.0   
9    10              Denmark          DEN          0.0             51.0   

   rank_change  cur_year_avg  cur_year_avg_weighted  last_year_avg  \
0            0           0.0                    0.0            0.0   
1            0           0.0                    0.0            0.0   
2            9           0.0                    0.0            0.0   
3            0           0.0                    0.0            0.0   
4            5           0.0                    0.0            0.0   
5            0           0.0                    0.0            0.0   
6            1           0.0                    0.0            0.0   
7           -5           0.0                    0.0            0.0   
8            5           0.0                    0.0            0.0   
9           -1           0.0                    0.0            0.0   

   last_year_avg_weighted  two_year_ago_avg  two_year_ago_weighted  \
0                     0.0               0.0                    0.0   
1                     0.0               0.0                    0.0   
2                     0.0               0.0                    0.0   
3                     0.0               0.0                    0.0   
4                     0.0               0.0                    0.0   
5                     0.0               0.0                    0.0   
6                     0.0               0.0                    0.0   
7                     0.0               0.0                    0.0   
8                     0.0               0.0                    0.0   
9                     0.0               0.0                    0.0   

   three_year_ago_avg  three_year_ago_weighted confederation   rank_date  \
0                 0.0                      0.0          UEFA  1993-08-08   
1                 0.0                      0.0          UEFA  1993-08-08   
2                 0.0                      0.0          UEFA  1993-08-08   
3                 0.0                      0.0          UEFA  1993-08-08   
4                 0.0                      0.0      CONMEBOL  1993-08-08   
5                 0.0                      0.0          UEFA  1993-08-08   
6                 0.0                      0.0          UEFA  1993-08-08   
7                 0.0                      0.0      CONMEBOL  1993-08-08   
8                 0.0                      0.0          UEFA  1993-08-08   
9                 0.0                      0.0          UEFA  1993-08-08   

  Unnamed: 16  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6         NaN  
7         NaN  
8         NaN  
9         NaN

In [9]:
ranking_2018 = fifa_ranking[(fifa_ranking['rank_date']>"2018-04-01") & (fifa_ranking['rank_date']<"2018-05-01")]
ranking_2014 = fifa_ranking[(fifa_ranking['rank_date']>"2014-04-01") & (fifa_ranking['rank_date']<"2014-05-01")]
ranking_2010 = fifa_ranking[(fifa_ranking['rank_date']>"2010-04-01") & (fifa_ranking['rank_date']<"2010-05-01")]
ranking_2006 = fifa_ranking[(fifa_ranking['rank_date']>"2006-04-01") & (fifa_ranking['rank_date']<"2006-05-01")]
ranking_2002 = fifa_ranking[(fifa_ranking['rank_date']>"2002-03-01") & (fifa_ranking['rank_date']<"2002-04-01")]
ranking_1998 = fifa_ranking[(fifa_ranking['rank_date']>"1998-04-01") & (fifa_ranking['rank_date']<"1998-05-01")]
ranking_1994 = fifa_ranking[(fifa_ranking['rank_date']>"1994-04-01") & (fifa_ranking['rank_date']<"1994-05-01")]


In [10]:
ranking_2018.drop_duplicates(keep='last',inplace=True)
ranking_2014.drop_duplicates(keep='last',inplace=True) 
ranking_2010.drop_duplicates(keep='last',inplace=True) 
ranking_2006.drop_duplicates(keep='last',inplace=True) 
ranking_2002.drop_duplicates(keep='last',inplace=True) 
ranking_1998.drop_duplicates(keep='last',inplace=True) 
ranking_1994.drop_duplicates(keep='last',inplace=True)

/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports unti

# add ranking to df

In [11]:
df.loc[df.Year == 2018,'rank_Team1'] = df['Team1'].map(ranking_2018.set_index('country_full')['rank'])
df.loc[df.Year == 2018,'rank_Team2'] = df['Team2'].map(ranking_2018.set_index('country_full')['rank'])

df.loc[df.Year == 2014,'rank_Team1'] = df['Team1'].map(ranking_2014.set_index('country_full')['rank'])
df.loc[df.Year == 2014,'rank_Team2'] = df['Team2'].map(ranking_2014.set_index('country_full')['rank'])

df.loc[df.Year == 2010,'rank_Team1'] = df['Team1'].map(ranking_2010.set_index('country_full')['rank'])
df.loc[df.Year == 2010,'rank_Team2'] = df['Team2'].map(ranking_2010.set_index('country_full')['rank'])

df.loc[df.Year == 2006,'rank_Team1'] = df['Team1'].map(ranking_2006.set_index('country_full')['rank'])
df.loc[df.Year == 2006,'rank_Team2'] = df['Team2'].map(ranking_2006.set_index('country_full')['rank'])

df.loc[df.Year == 2002,'rank_Team1'] = df['Team1'].map(ranking_2002.set_index('country_full')['rank'])
df.loc[df.Year == 2002,'rank_Team2'] = df['Team2'].map(ranking_2002.set_index('country_full')['rank'])

df.loc[df.Year == 1998,'rank_Team1'] = df['Team1'].map(ranking_1998.set_index('country_full')['rank'])
df.loc[df.Year == 1998,'rank_Team2'] = df['Team2'].map(ranking_1998.set_index('country_full')['rank'])

df.loc[df.Year == 1994,'rank_Team1'] = df['Team1'].map(ranking_1994.set_index('country_full')['rank'])
df.loc[df.Year == 1994,'rank_Team2'] = df['Team2'].map(ranking_1994.set_index('country_full')['rank'])

df.loc[df.Year == 1990,'rank_Team1'] = df['Team1'].map(ranking_1994.set_index('country_full')['rank'])
df.loc[df.Year == 1990,'rank_Team2'] = df['Team2'].map(ranking_1994.set_index('country_full')['rank'])

In [12]:
df.head()

Team1         Team2       Rounds                   Stadium  \
0    Russia  Saudi Arabia  Group Phase          Luzhniki Stadium   
1     Egypt       Uruguay  Group Phase        Ekaterinburg Arena   
2   Morocco       IR Iran  Group Phase  Saint Petersburg Stadium   
3  Portugal         Spain  Group Phase             Fisht Stadium   
4    France     Australia  Group Phase               Kazan Arena   

             City   Time   Day  Month    Year  Score1  Score2  \
0          Moscow  18:00  14.0    6.0  2018.0     5.0     0.0   
1    Ekaterinburg  17:00  15.0    6.0  2018.0     0.0     1.0   
2  St. Petersburg  18:00  15.0    6.0  2018.0     0.0     1.0   
3           Sochi  21:00  15.0    6.0  2018.0     3.0     3.0   
4           Kazan  13:00  16.0    6.0  2018.0     2.0     1.0   

                                             Remarks  rank_Team1  rank_Team2  
0                                                NaN        66.0        70.0  
1                                                NaN        46.0        17.0  
2                                                NaN        42.0        36.0  
3  {WinTeamName} win on aggregate after regular t...         4.0         8.0  
4                                                NaN         7.0        40.0

In [13]:
df.isnull().sum()

Team1           0
Team2           0
Rounds          0
Stadium         0
City            0
Time            0
Day             0
Month           0
Year            0
Score1          0
Score2          0
Remarks       866
rank_Team1      0
rank_Team2      0
dtype: int64

In [14]:
df[df.rank_Team1.isnull()]

Empty DataFrame
Columns: [Team1, Team2, Rounds, Stadium, City, Time, Day, Month, Year, Score1, Score2, Remarks, rank_Team1, rank_Team2]
Index: []

# Team Feature

In [15]:
df_teams.head()

player  Year       team  goals_z  xg_z  crosses_z  boxtouches_z  \
0  Cristian Pavón  2018  Argentina    -0.38 -0.51       0.08         -0.25   
1  Eduardo Salvio  2018  Argentina    -0.38 -0.48      -0.50          0.62   
2      Enzo Pérez  2018  Argentina    -0.38  0.05      -0.50         -0.69   
3  Federico Fazio  2018  Argentina    -0.38 -0.55      -0.50         -0.69   
4   Franco Armani  2018  Argentina    -0.38 -0.55      -0.50         -0.80   

   passes_z  progpasses_z  takeons_z  progruns_z  tackles_z  interceptions_z  \
0     -0.65         -0.74      -0.08        0.14       0.24            -0.77   
1     -0.03         -0.49      -0.08        0.46       0.52             0.59   
2      0.20         -0.68      -0.39       -0.35       0.28            -0.24   
3     -1.00         -0.93      -0.70       -0.84      -0.80            -0.77   
4     -0.70          0.60      -0.70       -0.68      -0.80            -0.77   

   clearances_z  blocks_z  aerials_z  fouls_z  fouled_z  nsxg_z  
0         -0.81     -0.44      -0.24    -0.40     -0.93   -0.60  
1          0.32     -0.44       0.19    -1.12      0.62   -0.05  
2         -0.81     -0.44      -0.89     0.68      0.00   -0.68  
3         -0.65     -0.44       0.19    -1.12     -0.62   -0.79  
4         -0.65     -0.44      -0.89    -1.12     -0.93   -0.79

In [16]:
features=[]
for key in df_teams.keys():
    features.append(str(key))

In [17]:
features

['player',
 'Year',
 'team',
 'goals_z',
 'xg_z',
 'crosses_z',
 'boxtouches_z',
 'passes_z',
 'progpasses_z',
 'takeons_z',
 'progruns_z',
 'tackles_z',
 'interceptions_z',
 'clearances_z',
 'blocks_z',
 'aerials_z',
 'fouls_z',
 'fouled_z',
 'nsxg_z']

In [18]:
del features[0:4]

In [19]:
features

['xg_z',
 'crosses_z',
 'boxtouches_z',
 'passes_z',
 'progpasses_z',
 'takeons_z',
 'progruns_z',
 'tackles_z',
 'interceptions_z',
 'clearances_z',
 'blocks_z',
 'aerials_z',
 'fouls_z',
 'fouled_z',
 'nsxg_z']

In [20]:
df_teams_avg = df_teams.groupby(['Year','team'])['goals_z'].sum()/df_teams.groupby(['Year','team'])['goals_z'].size()
df_teams_avg = df_teams_avg.to_frame()

In [21]:
df_teams_avg['Year'] = df_teams_avg.index.get_level_values(0).tolist()
df_teams_avg['team'] = df_teams_avg.index.get_level_values(1).tolist()


In [22]:
df_teams_avg.head()

goals_z  Year       team
Year team                                
1966 Argentina  0.049167  1966  Argentina
     Brazil    -0.038000  1966     Brazil
     Bulgaria  -0.280000  1966   Bulgaria
     Chile     -0.139286  1966      Chile
     England    0.221333  1966    England

In [23]:
def avg_team(feature,df_teams_avg):
    a=df_teams.groupby(['Year','team'])[feature].sum()/df_teams.groupby(['Year','team'])[feature].size()
    b=a.to_frame()
    df_teams_avg[feature]= b[feature]
    

In [24]:
for feature in features:
    avg_team(feature,df_teams_avg)

In [25]:
df_teams_avg.head()

goals_z  Year       team      xg_z  crosses_z  boxtouches_z  \
Year team                                                                      
1966 Argentina  0.049167  1966  Argentina  0.021667  -0.440833     -0.138333   
     Brazil    -0.038000  1966     Brazil -0.185500  -0.249500     -0.097000   
     Bulgaria  -0.280000  1966   Bulgaria -0.206875  -0.335000     -0.199375   
     Chile     -0.139286  1966      Chile  0.160714  -0.050000      0.007857   
     England    0.221333  1966    England  0.016000   0.424000      0.221333   

                passes_z  progpasses_z  takeons_z  progruns_z  tackles_z  \
Year team                                                                  
1966 Argentina  0.836667      0.265000   0.621667    0.572500   0.090000   
     Brazil    -0.145500     -0.179000   0.139500   -0.237500   0.017000   
     Bulgaria  -0.128125     -0.018125   0.344375   -0.025000  -0.235000   
     Chile      0.515714      0.370714   0.130714    0.074286   0.407143   
     England    0.019333     -0.035333   0.048667    0.274667  -0.225333   

                interceptions_z  clearances_z  blocks_z  aerials_z   fouls_z  \
Year team                                                                      
1966 Argentina         0.501667      0.219167 -0.064167  -0.134167  0.522500   
     Brazil            0.039000     -0.275500 -0.229000  -0.255500 -0.141500   
     Bulgaria         -0.583125     -0.096250 -0.108125  -0.302500 -0.072500   
     Chile             0.215714     -0.045714  0.260000  -0.259286  0.090000   
     England          -0.218667      0.046667  0.160667   0.450000  0.189333   

                fouled_z    nsxg_z  
Year team                           
1966 Argentina  0.565833 -0.110833  
     Brazil    -0.240500 -0.016500  
     Bulgaria   0.029375 -0.013750  
     Chile      0.104286  0.019286  
     England   -0.310667  0.136000

In [26]:
new_df = pd.merge(df, df_teams_avg,  how='left', left_on=['Team1','Year'], right_on = ['team','Year'])
new_df1 = pd.merge(new_df,df_teams_avg ,  how='left', left_on=['Team2','Year'], right_on = ['team','Year'])


In [27]:
new_df1.head()

Team1         Team2       Rounds                   Stadium  \
0    Russia  Saudi Arabia  Group Phase          Luzhniki Stadium   
1     Egypt       Uruguay  Group Phase        Ekaterinburg Arena   
2   Morocco       IR Iran  Group Phase  Saint Petersburg Stadium   
3  Portugal         Spain  Group Phase             Fisht Stadium   
4    France     Australia  Group Phase               Kazan Arena   

             City   Time   Day  Month  Year  Score1    ...     takeons_z_y  \
0          Moscow  18:00  14.0    6.0  2018     5.0    ...       -0.136316   
1    Ekaterinburg  17:00  15.0    6.0  2018     0.0    ...       -0.080625   
2  St. Petersburg  18:00  15.0    6.0  2018     0.0    ...       -0.106250   
3           Sochi  21:00  15.0    6.0  2018     3.0    ...        0.428235   
4           Kazan  13:00  16.0    6.0  2018     2.0    ...        0.097333   

  progruns_z_y  tackles_z_y  interceptions_z_y  clearances_z_y blocks_z_y  \
0     0.073684     0.512632           0.126316       -0.246316   0.209474   
1     0.025000    -0.043750           0.891875        0.015000  -0.056250   
2    -0.435625    -0.701250          -0.608750        0.289375  -0.327500   
3     0.800000     0.513529           0.456471       -0.374706  -0.008824   
4    -0.029333     0.031333          -0.022667        0.221333  -0.341333   

   aerials_z_y  fouls_z_y  fouled_z_y  nsxg_z_y  
0    -0.466316  -0.387368    0.051579 -0.294211  
1    -0.066875  -0.130000    0.116250  0.041875  
2    -0.082500   0.200000    0.236250 -0.388125  
3    -0.140000  -0.400000    0.145882  0.644706  
4     0.240667   0.096000   -0.071333  0.004000  

[5 rows x 48 columns]

In [28]:
new_df1.iloc[0]

Team1                          Russia
Team2                    Saudi Arabia
Rounds                    Group Phase
Stadium              Luzhniki Stadium
City                           Moscow
Time                            18:00
Day                                14
Month                               6
Year                             2018
Score1                              5
Score2                              0
Remarks                           NaN
rank_Team1                         66
rank_Team2                         70
goals_z_x                       0.315
team_x                         Russia
xg_z_x                      -0.104375
crosses_z_x                 -0.033125
boxtouches_z_x               -0.06875
passes_z_x                  -0.315625
progpasses_z_x              -0.049375
takeons_z_x                 -0.176875
progruns_z_x                -0.159375
tackles_z_x                    -0.375
interceptions_z_x            -0.12875
clearances_z_x               0.246875
blocks_z_x  

In [29]:
df =new_df1

In [30]:
df.isnull().sum()

Team1                  0
Team2                  0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              866
rank_Team1             0
rank_Team2             0
goals_z_x             49
team_x                49
xg_z_x                49
crosses_z_x           49
boxtouches_z_x        49
passes_z_x            49
progpasses_z_x        49
takeons_z_x           49
progruns_z_x          49
tackles_z_x           49
interceptions_z_x     49
clearances_z_x        49
blocks_z_x            49
aerials_z_x           49
fouls_z_x             49
fouled_z_x            49
nsxg_z_x              49
goals_z_y             49
team_y                49
xg_z_y                49
crosses_z_y           49
boxtouches_z_y        49
passes_z_y            49
progpasses_z_y        49
takeons_z_y           49
progruns_z_y          49


In [31]:
df[df.goals_z_x.isnull()]

Team1                  Team2          Rounds  \
71            Côte dIvoire                  Japan     Group Phase   
140           Côte dIvoire               Portugal     Group Phase   
173              Korea DPR           Côte dIvoire     Group Phase   
197  Serbia and Montenegro            Netherlands     Group Phase   
231           Côte dIvoire  Serbia and Montenegro     Group Phase   
258    Republic of Ireland               Cameroon     Group Phase   
412    Republic of Ireland                 Norway     Group Phase   
437                 Russia                Romania     Group Phase   
442                Germany             Yugoslavia     Group Phase   
453                 Mexico    Republic of Ireland     Group Phase   
457           Saudi Arabia                Morocco     Group Phase   
462                Germany         Korea Republic     Group Phase   
464    Republic of Ireland                 Norway     Group Phase   
470    Republic of Ireland            Netherlands     Group Phase   
473         Czech Republic             Costa Rica     Round of 16   
474           Saudi Arabia                 Sweden     Round of 16   
475                Germany            Netherlands     Round of 16   
482                Germany         Czech Republic  Quarter Finals   
485                Germany                England     Semi Finals   
487                Germany              Argentina          Finals   
572           Côte dIvoire               Colombia     Group Phase   
591           Côte dIvoire                 Greece     Group Phase   
629              Korea DPR                 Brazil     Group Phase   
644           Côte dIvoire                 Brazil     Group Phase   
645              Korea DPR               Portugal     Group Phase   
661           Côte dIvoire              Korea DPR     Group Phase   
684           Côte dIvoire              Argentina     Group Phase   
700  Serbia and Montenegro              Argentina     Group Phase   
701           Côte dIvoire            Netherlands     Group Phase   
719  Serbia and Montenegro           Côte dIvoire     Group Phase   
760    Republic of Ireland                Germany     Group Phase   
778    Republic of Ireland           Saudi Arabia     Group Phase   
794    Republic of Ireland                  Spain     Round of 16   
875    Republic of Ireland                  Italy     Group Phase   
889    Republic of Ireland                 Mexico     Group Phase   
912    Republic of Ireland            Netherlands     Round of 16   
927    Republic of Ireland                  Italy     Group Phase   
928         Czech Republic                    USA     Group Phase   
932           Saudi Arabia            Netherlands     Group Phase   
936                 Russia              Argentina     Group Phase   
940                Bolivia         Korea Republic     Group Phase   
941    Republic of Ireland                 Mexico     Group Phase   
945                Morocco           Saudi Arabia     Group Phase   
949                 Russia               Cameroon     Group Phase   
952                 Norway    Republic of Ireland     Group Phase   
957           Saudi Arabia                Belgium     Group Phase   
964    Republic of Ireland            Netherlands     Round of 16   
969    Republic of Ireland                  Italy  Quarter Finals   
970         Czech Republic                Germany  Quarter Finals   

                                   Stadium                               City  \
71                        Arena Pernambuco                             Recife   
140                 Port Elizabeth Stadium  Nelson Mandela Bay/Port Elizabeth   
173                       Mbombela Stadium                          Nelspruit   
197                         Zentralstadion                            Leipzig   
231         FIFA World Cup Stadium, Munich                             Munich   
258               Niigata Stadium Big Swan                            Niigata   
412

In [32]:
df.head()

Team1         Team2       Rounds                   Stadium  \
0    Russia  Saudi Arabia  Group Phase          Luzhniki Stadium   
1     Egypt       Uruguay  Group Phase        Ekaterinburg Arena   
2   Morocco       IR Iran  Group Phase  Saint Petersburg Stadium   
3  Portugal         Spain  Group Phase             Fisht Stadium   
4    France     Australia  Group Phase               Kazan Arena   

             City   Time   Day  Month  Year  Score1    ...     takeons_z_y  \
0          Moscow  18:00  14.0    6.0  2018     5.0    ...       -0.136316   
1    Ekaterinburg  17:00  15.0    6.0  2018     0.0    ...       -0.080625   
2  St. Petersburg  18:00  15.0    6.0  2018     0.0    ...       -0.106250   
3           Sochi  21:00  15.0    6.0  2018     3.0    ...        0.428235   
4           Kazan  13:00  16.0    6.0  2018     2.0    ...        0.097333   

  progruns_z_y  tackles_z_y  interceptions_z_y  clearances_z_y blocks_z_y  \
0     0.073684     0.512632           0.126316       -0.246316   0.209474   
1     0.025000    -0.043750           0.891875        0.015000  -0.056250   
2    -0.435625    -0.701250          -0.608750        0.289375  -0.327500   
3     0.800000     0.513529           0.456471       -0.374706  -0.008824   
4    -0.029333     0.031333          -0.022667        0.221333  -0.341333   

   aerials_z_y  fouls_z_y  fouled_z_y  nsxg_z_y  
0    -0.466316  -0.387368    0.051579 -0.294211  
1    -0.066875  -0.130000    0.116250  0.041875  
2    -0.082500   0.200000    0.236250 -0.388125  
3    -0.140000  -0.400000    0.145882  0.644706  
4     0.240667   0.096000   -0.071333  0.004000  

[5 rows x 48 columns]

# Index the Rounds

In [33]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}
rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
    
df["Rounds"] = rounds_encoded

In [34]:
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
    
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [35]:
#Adding who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append(68)
df['winning_team'] = winner

df.head()

Team1  Team2  Rounds                   Stadium            City   Time  \
0     36     27       1          Luzhniki Stadium          Moscow  18:00   
1     50     30       1        Ekaterinburg Arena    Ekaterinburg  17:00   
2     33     46       1  Saint Petersburg Stadium  St. Petersburg  18:00   
3      3     42       1             Fisht Stadium           Sochi  21:00   
4      7     35       1               Kazan Arena           Kazan  13:00   

    Day  Month  Year  Score1      ...       progruns_z_y tackles_z_y  \
0  14.0    6.0  2018     5.0      ...           0.073684    0.512632   
1  15.0    6.0  2018     0.0      ...           0.025000   -0.043750   
2  15.0    6.0  2018     0.0      ...          -0.435625   -0.701250   
3  15.0    6.0  2018     3.0      ...           0.800000    0.513529   
4  16.0    6.0  2018     2.0      ...          -0.029333    0.031333   

   interceptions_z_y  clearances_z_y  blocks_z_y aerials_z_y  fouls_z_y  \
0           0.126316       -0.246316    0.209474   -0.466316  -0.387368   
1           0.891875        0.015000   -0.056250   -0.066875  -0.130000   
2          -0.608750        0.289375   -0.327500   -0.082500   0.200000   
3           0.456471       -0.374706   -0.008824   -0.140000  -0.400000   
4          -0.022667        0.221333   -0.341333    0.240667   0.096000   

   fouled_z_y  nsxg_z_y  winning_team  
0    0.051579 -0.294211            36  
1    0.116250  0.041875            30  
2    0.236250 -0.388125            46  
3    0.145882  0.644706            68  
4   -0.071333  0.004000             7  

[5 rows x 49 columns]

In [36]:
df.isnull().sum()

Team1                  0
Team2                  0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              866
rank_Team1             0
rank_Team2             0
goals_z_x             49
team_x                49
xg_z_x                49
crosses_z_x           49
boxtouches_z_x        49
passes_z_x            49
progpasses_z_x        49
takeons_z_x           49
progruns_z_x          49
tackles_z_x           49
interceptions_z_x     49
clearances_z_x        49
blocks_z_x            49
aerials_z_x           49
fouls_z_x             49
fouled_z_x            49
nsxg_z_x              49
goals_z_y             49
team_y                49
xg_z_y                49
crosses_z_y           49
boxtouches_z_y        49
passes_z_y            49
progpasses_z_y        49
takeons_z_y           49
progruns_z_y          49


In [37]:
df = df.dropna(axis=0, subset=['goals_z_x','goals_z_y'])
df.isnull().sum()

Team1                  0
Team2                  0
Rounds                 0
Stadium                0
City                   0
Time                   0
Day                    0
Month                  0
Year                   0
Score1                 0
Score2                 0
Remarks              790
rank_Team1             0
rank_Team2             0
goals_z_x              0
team_x                 0
xg_z_x                 0
crosses_z_x            0
boxtouches_z_x         0
passes_z_x             0
progpasses_z_x         0
takeons_z_x            0
progruns_z_x           0
tackles_z_x            0
interceptions_z_x      0
clearances_z_x         0
blocks_z_x             0
aerials_z_x            0
fouls_z_x              0
fouled_z_x             0
nsxg_z_x               0
goals_z_y              0
team_y                 0
xg_z_y                 0
crosses_z_y            0
boxtouches_z_y         0
passes_z_y             0
progpasses_z_y         0
takeons_z_y            0
progruns_z_y           0


In [38]:
df.shape

(890, 49)

In [39]:
df.Team1.max()

67

# train & test

In [40]:
df = df.drop(['Score1','Score2','Remarks',"Stadium","City",'team_x','team_y'],axis=1)
df = df.fillna(0)
df.head()

Team1  Team2  Rounds   Time   Day  Month    Year  rank_Team1  rank_Team2  \
0     36     27       1  18:00  14.0    6.0  2018.0        66.0        70.0   
1     50     30       1  17:00  15.0    6.0  2018.0        46.0        17.0   
2     33     46       1  18:00  15.0    6.0  2018.0        42.0        36.0   
3      3     42       1  21:00  15.0    6.0  2018.0         4.0         8.0   
4      7     35       1  13:00  16.0    6.0  2018.0         7.0        40.0   

   goals_z_x      ...       progruns_z_y  tackles_z_y  interceptions_z_y  \
0   0.315000      ...           0.073684     0.512632           0.126316   
1  -0.148750      ...           0.025000    -0.043750           0.891875   
2  -0.264375      ...          -0.435625    -0.701250          -0.608750   
3   0.110000      ...           0.800000     0.513529           0.456471   
4   0.037000      ...          -0.029333     0.031333          -0.022667   

   clearances_z_y  blocks_z_y  aerials_z_y  fouls_z_y  fouled_z_y  nsxg_z_y  \
0       -0.246316    0.209474    -0.466316  -0.387368    0.051579 -0.294211   
1        0.015000   -0.056250    -0.066875  -0.130000    0.116250  0.041875   
2        0.289375   -0.327500    -0.082500   0.200000    0.236250 -0.388125   
3       -0.374706   -0.008824    -0.140000  -0.400000    0.145882  0.644706   
4        0.221333   -0.341333     0.240667   0.096000   -0.071333  0.004000   

   winning_team  
0            36  
1            30  
2            46  
3            68  
4             7  

[5 rows x 42 columns]

In [41]:
# X_train = df.loc[(df['Year']!= 2018) | ((df['Year'] == 2018) & (df['Rounds'] ==1))]
# X_test  = df.loc[(df['Year'] == 2018) & (df['Rounds'] != 1)]

In [42]:
# y_train =  X_train["winning_team"]
# y_test  =  X_test["winning_team"]

In [43]:
# X_train = X_train.drop(["Rounds","winning_team","Time"],axis=1)
# X_test = X_test.drop(["Rounds","winning_team","Time"],axis=1)

In [44]:
# from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()  
# X_train = sc.fit_transform(X_train)  
# X_test = sc.transform(X_test) 

# XGboost

In [45]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

NameError: name 'X_train' is not defined

In [51]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

NameError: name 'X_train' is not defined

In [ ]:
# param = {
#     'max_depth': 50,  # the maximum depth of each tree
#     'eta': 0.05,  # the training step for each iteration
#     'silent': 1,  # logging mode - quiet
#     'objective': 'multi:softprob',  # error evaluation for multiclass training
#     'num_class': 69}  # the number of classes that exist in this datset
# num_round = 1000  # the number of training iterations

In [ ]:
# bst = xgb.train(param, dtrain, num_round)
# preds = bst.predict(dtest)

In [ ]:
# import numpy as np
# best_preds = np.asarray([np.argmax(line) for line in preds])


In [ ]:
# from sklearn.metrics import precision_score

# print( precision_score(y_test, best_preds, average='macro'))
# print(accuracy_score(y_test, best_preds))

# NEw

# SVM

In [ ]:
from sklearn import svm
model_svm = svm.SVC()
model_svm.fit(X_train,y_train)

y_pred_svm = model_svm.predict(X_test)

accuracy_score(y_test, y_pred_svm)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model= DecisionTreeClassifier()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

y_pred_knn = neigh.predict(X_test)

accuracy_score(y_test, y_pred_knn)

# NN

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?

model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=64)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))